<a href="https://colab.research.google.com/github/ucfilho/raianars_fly/blob/master/RTC_DE_desmonta_IF_new_12_fev_22_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x,cols): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x,cols):

  nref=int(len(x)/2) # metade de x é deltaT e a outra metade variavel logica
  matrizDeltaT=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  contador=0
  for i in range(cols):
    for j in range(cols):
      soma=contador+nref
      #print('%d %d %d %d %d'%(i,j,contador,nref,soma))
      flag=0
      if(i<j):
        if(contador<nref):
          flag=1
      if(flag>0):
        matrizDeltaT[i,j]=x[contador]
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
        
  return matrizDeltaT,matrizBinaria

In [0]:
def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):

  cols=len(mCp)
  cargaTermica=np.zeros((cols,cols))
  contador=0
  erro=0 # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for i in range(cols):
    for j in range(cols):
      contador=contador+1
      cargaTermica[i,j]=mCp[i]*matrizDeltaT[i,j]

      if(i<j):
        if(matrizBinaria[i,j]==1):
          #i: CQ, j: CF
          tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaF=-cargaTermica[i,j]/(mCp[j])+tempAtual[j]
          flag=0 # flag : indica que nao entra no teste logico
          # usei flag no lugar q ifs pq a identacao ficou melhor assim
          if(tempAtual[i] > tempAtual[j]): #Tquente maior Tfria: vaores atuais
            if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
              if(contador < cols):
                flag=1
          
          if(flag>0):
            tempAtual[i]=tempSaidaQ
            tempAtual[j]=tempSaidaF
          else:
            erro=1


  return tempAtual,cargaTermica,erro

In [0]:
def Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT):
  cols=len(tempInicial)
  Trocador=np.zeros((cols,cols)) # matriz q armazena as areas dos trocadores
  Soma_Area=0

  for i in range(cols):
    for j in range(cols):
      if(i<j):
        Carga=cargaTermica[i,j]
        # calcula a area de placas das duas correntes que trocam i e j

        #a corrente i perde calor para a j
        #i: CQ, j:CF
        tempSaidaF=tempAtual[j]
        tempSaidaQ=tempAtual[i]
        try:
          A=((tempSaidaQ-tempSaidaF)-(tempAtual[i]-tempAtual[j]))
          B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[i]-tempAtual[j]))
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99
 
        if(deltaTMediaLog==1e99):  
          AreaCalc=1e99
        else:
          AreaCalc=abs(Carga/(U*deltaTMediaLog))

        if(tempAtual[i]==tempInicial[i]):
          AreaCalc=0

        if(tempAtual[j]==tempInicial[j]):
          AreaCalc=0

        Soma_Area=Soma_Area+AreaCalc

        Trocador[i,j]=AreaCalc

  return Soma_Area,Trocador

In [0]:
def Utilidades(tempAlvo,tempAtual,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  areaTrocador=0
  Trocadores=np.zeros(cols)
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353
  for i in range(cols):
    deltaT=tempAlvo[i]-tempAtual[i]
    if(deltaT>0):#usa UQ
      cargaUQ=cargaUQ+mCp[i]*deltaT
      deltaT1=(TUQout-tempAlvo[i])
      deltaT2=(TUQin-tempAtual[i])

      try:

        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      areaTrocador=areaTrocador+abs(mCp[i]*(deltaT)/(U*deltaTMediaLog))
    elif(deltaT==0):#nao usa UF e UQ
      deltaTMediaLog=0      
    else:#usa UF
      deltaT1=(tempAlvo[i]-TUFout)
      deltaT2=(tempAtual[i]-TUFin)     
      cargaUF=cargaUF+mCp[i]*(-deltaT)

      #UF -> água resfriada -> entra a 5ºC e sai a 20ºC
      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      
      if(deltaTMediaLog==1e99):
        areaTrocador=1e99
        Trocadores[i]=1e99
      else:
        Trocadores[i]=abs(mCp[i]*(-deltaT)/(U*deltaTMediaLog))
        areaTrocador=areaTrocador+Trocadores[i]

  return areaTrocador, Trocadores, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  
  cols=len(mCp)
  tempInicial=np.copy(tempAtual) 
  matrizDeltaT,matrizBinaria=buildMatriz(x,cols) # transforma x em matriz logica e troca
  tempAtual,Carga,Erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
  areaTrocador,matrizArea =Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
  areaUtilidade, TrocaUtilidade, cargaUF, cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)

  costTrocador=300*(areaTrocador+areaUtilidade)**0.5

  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF

  cost=costUQ+costUF+costTrocador

  funr = np.where(np.isnan(cost), 1e99, cost)
  
  if(Erro>0):
    funr=1e99


  return funr

In [0]:
#MATGRAF retorna o gráfico das trocas térmicas em cada trocador
# function matrizGrafico=MATGRAF(matrizDeltaT,mCp,tempAtual)
def MATGRAF(matrizDeltaT,mCp,tempAtual):
  cols=len(mCp)
  matrizGrafico=np.zeros((cols,cols))
  contador=0
  matrizGrafico[:,0]=tempAtual
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        contador=contador+1
        # i: CQ, j:CF
        cargaTermica=mCp[i]*matrizDeltaT[i,j]
        tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
        tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]

        flag=0 # flag : indica que nao entra no teste logico
        # usei flag no lugar ifs pq identacao ficou melhor 
        if(tempAtual[i] > tempAtual[j]): #Tquente maior Tfria: valores atuais
          if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
            if(contador < cols):
              flag=1
        

        if(flag>0):
          tempAtual[i]=tempSaidaQ
          tempAtual[j]=tempSaidaF
          matrizGrafico[:,contador]=tempAtual
          

  return matrizGrafico

In [0]:
#MATBEST retorna a matriz com os deltas das trocas térmicas que ocorreram e a matriz binária
# function [matrizTrocaBest,matrizBinaria]=MATBEST(x,mCp,tempAtual)
def MATBEST(x,mCp,tempAtual):
  
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)

  # cada particula contem metade das variaveis para matriz binaria
  # e outra metade para deltaT nref e para separar os dois casos
  matrizDeltaT=np.zeros((cols,cols))
  matrizTrocaBest=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  # contador=-1
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        # contador=contador+1
        matrizDeltaT[i,j]=x[contador]
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
    
  
    tempSaidaQ=0;tempSaidaF=0;cargaTermica=0
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          if(matrizBinaria[i,j]==1):
            if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j 
              #i: CQ, j: CF
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]>tempAtual[j] and
                  tempSaidaQ>tempAtual[j] and
                  tempAtual[i]>tempSaidaF
                ): #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
                tempAtual[i]=tempSaidaQ
                tempAtual[j]=tempSaidaF
              else:
                matrizDeltaT[i,j]=0
            else:#a corrente i ganha calor da j
              # i: CF, j:CQ
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]<tempAtual[j] and
                  tempSaidaF<tempAtual[j] and
                  tempAtual[i]<tempSaidaQ
                 ):
                #TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
                tempAtual[i]=tempSaidaF
                tempAtual[j]=tempSaidaQ
              else:
                matrizDeltaT[i,j]=0
          else:
            matrizDeltaT[i,j]=0
  #print('contador=',contador)
  matrizTrocaBest=matrizDeltaT
  return matrizTrocaBest,matrizBinaria

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=100 #Número de partículas
ITE=1000 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
MIN=[-100,-100,-100,-100,-100,-100,0,0,0,0,0,0]
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))



In [11]:

fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n'%(ITE,NPAR));print(GBEST)
print('Valor da função f(gbest)=%f \n'%BEST)
print('matriz grafico');print(matrizGrafico)
#print('Matriz de Trocas Térmicas \n')
#print(matrizTrocaBest)
#print('Matriz Binária \n')
#print(matrizBinaria)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


temperatura atual= [433.1638186468277, 432.6723627063445, 523, 533]
[-8.32998395e+01 -7.01271313e+01 -3.77428358e+01 -2.44679814e+01
 -1.23951796e+01 -4.10584674e+01  2.78228169e-02  2.04409927e-01
  3.66514184e-01  6.52172838e-03  3.60751219e-01  6.66139102e-02] 59687.85608344324
[433.1638186468277, 432.6723627063445, 523, 533]
Melhor solução para 1000 iterações com 100 partículas 

[-8.32998395e+01 -7.01271313e+01 -3.77428358e+01 -2.44679814e+01
 -1.23951796e+01 -4.10584674e+01  2.78228169e-02  2.04409927e-01
  3.66514184e-01  6.52172838e-03  3.60751219e-01  6.66139102e-02]
Valor da função f(gbest)=59687.856083 

matriz grafico
[[433.16381865 433.16381865   0.           0.        ]
 [432.67236271 432.67236271   0.           0.        ]
 [523.         523.           0.           0.        ]
 [533.         533.           0.           0.        ]]


In [12]:
cols=len(mCp)
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
tempAtual,cargaTermica,erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
Soma_Area,Trocador=Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT)

print('temperatura inicial');print(tempInicial)
print('temperatura alvo');print(tempAlvo)
print('temperatura atual');print(tempAtual)
print('trocador');print(Trocador)

temperatura inicial
[453, 393, 523, 533]
temperatura alvo
[513, 508, 403, 433]
temperatura atual
[433.1638186468277, 432.6723627063445, 523, 533]
trocador
[[ 0. nan  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
print(cols)
print(GBEST)

4
[-8.32998395e+01 -7.01271313e+01 -3.77428358e+01 -2.44679814e+01
 -1.23951796e+01 -4.10584674e+01  2.78228169e-02  2.04409927e-01
  3.66514184e-01  6.52172838e-03  3.60751219e-01  6.66139102e-02]


In [14]:
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
print('Matriz de Trocas Térmicas \n')
print(matrizDeltaT)
print('Matriz Binária \n')
print(matrizBinaria)

Matriz de Trocas Térmicas 

[[  0.         -83.29983949 -70.12713131 -37.74283577]
 [  0.           0.         -24.46798143 -12.39517956]
 [  0.           0.           0.         -41.05846741]
 [  0.           0.           0.           0.        ]]
Matriz Binária 

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
